In [ ]:
lin_reg_cv = lm.LinearRegression()
cv_scores = cross_val_score(lin_reg_cv, X_non_norm, Y, cv=10)
print(np.mean(cv_scores))
print(np.std(cv_scores))
print(cv_scores)

In [ ]:
v = np.linalg.inv(X.T.dot(X).values)

In [ ]:
def sse_calc(y, y_hat):
    return np.sum(np.square(y - y_hat))

In [ ]:
def z_calc(beta, n, k, v_j, sse):
    sigma = np.sqrt(1/(n-k-1) * sse)
    return beta/(sigma * np.sqrt(v_j))

In [ ]:
kf = KFold(n_splits=10)
kf.split(X)
n_fold = 1
p = len(X.columns)
z_all = np.empty((0,p))
print(np.shape(z_all))

for train_index, test_index in kf.split(X):
    z_row = []
    train_fold_X = X_non_norm.iloc[train_index]
    train_fold_Y = Y.iloc[train_index]

    test_fold_X = X_non_norm.iloc[test_index]
    test_fold_Y = Y.iloc[test_index]

    lin_reg_fold = lm.LinearRegression().fit(train_fold_X, train_fold_Y)
    pred_fold = lin_reg_fold.predict(test_fold_X)

    n = len(train_fold_X) ### should be test length
    k = len(train_fold_X.columns)
    xtx = train_fold_X.T.dot(train_fold_X).values
    v = np.linalg.pinv(xtx)

    sse = sse_calc(test_fold_Y, pred_fold)
    beta = lin_reg_fold.coef_

    for j in range(len(beta)):
        v_j = v[j,j]
        b_j = beta[j]
        z_j = z_calc(b_j, n, k, v_j, sse)
        z_row.append(z_j)
    z_all = np.append(z_all, np.array([z_row]), axis=0)
    n_fold += 1

In [ ]:
z_all = pd.DataFrame(z_all, columns=columns)
z_all.describe()

In [ ]:
# z_all.info(verbose=True, show_counts=True)

In [ ]:
array = np.empty((0,3))

array = np.append(array, np.array([[1,3,5]]), axis=0)
array = np.append(array, np.array([[2,4,6]]), axis=0)

In [ ]:
X_sm = sm.add_constant(X_non_norm)
sm_OLS_model = OLS(Y,X_sm)
result_sm = sm_OLS_model.fit()
a = result_sm.tvalues.to_frame().abs()
a.loc['const',0]
# X_test_sm = sm.add_constant(X_test, has_constant='add')
# Y_sm_pred = result_sm.predict(sm.add_constant(X_test_sm))

In [ ]:
result_sm.summary()
t_df = result_sm.tvalues.to_frame().abs()
t_df.sort_values(by=[0]).loc['const']

In [ ]:
result_sm.tvalues['const'] = np.inf
result_sm.tvalues


In [ ]:
BSR_reg = lm.LinearRegression()
param_grid = {'fit_intercept': [True]}
BSR_grid_cv = ms.GridSearchCV(BSR_reg, param_grid, cv=10)
BSR_grid_cv.fit(X_non_norm, Y)
pd.DataFrame(BSR_grid_cv.cv_results_)

